In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plot
import numpy as np
from numpy import genfromtxt
import csv
import sys, os, re
import pathlib

print(tf.__version__)

2.0.0


In [3]:
# The CANlab/WAVi Pain Study is Pre-Formatted to resemble BIDS neuroimaging formatting
# If your study does not abide to the following structure, please revisit previous scripts
# .../StudyRepo
# -------------> /raw
# -------------------> /*.art
# -------------------> /*.eeg
# -------------------> /*.evt
# -------------> /contigs
# -------------------> /train          2:1 train:test
# -------------------> /test

In [4]:
if "contigs" not in os.getcwd():
    try:
        os.chdir("CANlabStudy/contigs")
        directory=os.getcwd()
    except:
        print("I couldn't find the contigs folder.\n")
        directory = input("Please give the full path of the contigs folder: ")
        os.chdir(directory)

In [20]:
train_path = pathlib.Path('train/')
#train_path = pathlib.Path('train/p300/')
test_path = pathlib.Path('test/')
#test_path = pathlib.Path('test/p300/')

In [21]:
# load in image paths for training set

import random
train_image_paths = list(train_path.glob('*/*'))
train_image_paths = os.listdir(train_path)
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)

train_count = len(train_image_paths)
print("You have", train_count, "training images.")

# load in image paths for testing set?

test_image_paths = list(test_path.glob('*/*'))
test_image_paths = os.listdir(test_path)
test_image_paths = [str(path) for path in test_image_paths]
#test_image_paths = [str(path) for path in test_image_paths if "ctrl" in path]
random.shuffle(test_image_paths)

test_count = len(test_image_paths)
print("You have", test_count, "testing images.")

You have 2 training images.
You have 2 testing images.


In [9]:
#list the available labels
#label_names = sorted(item.name for item in train_path.glob('*/') if item.is_dir())
label_names = ['ctrl', 'pain']
print("Labels discovered:", label_names)

Labels discovered: ['ctrl', 'pain']


In [10]:
#assign an index to each label
label_to_index = dict((name, index) for index, name in enumerate(label_names))
print("Label indices:", label_to_index)

Label indices: {'ctrl': 0, 'pain': 1}


In [11]:
#create a list of every file and its index label
#train_image_labels = [label_to_index[pathlib.Path(path).parent.name]
 #                    for path in train_image_paths]

train_image_labels = [label_to_index[path[:4]] for path in train_image_paths]
    
#test_image_labels = [label_to_index[pathlib.Path(path).parent.name]
 #                   for path in test_image_paths]
    
test_image_labels = [label_to_index[path[:4]] for path in test_image_paths]

In [12]:
def load_numpy_stack(lead, paths):
    numpy_stack = []
    for path in paths:
        path = lead+path
        array = genfromtxt(path, delimiter=",")
        array = array.reshape(array.shape +(1,))
        numpy_stack.append(array)
    return(numpy_stack)

In [13]:
train_arrays = load_numpy_stack('train/p300/', train_image_paths)
test_arrays = load_numpy_stack('test/p300/', test_image_paths)

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_arrays, train_image_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_arrays, test_image_labels))

In [15]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [16]:
def kerasModel(learn, beta1, beta2):
    model = tf.keras.Sequential([
        tf.keras.layers.Convolution2D(64, kernel_size=3, strides=3, padding="same", dilation_rate=1, activation="relu", data_format="channels_last", use_bias=False, input_shape=(750, 19, 1)),
        #tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=None, padding="same"),
        tf.keras.layers.Flatten(input_shape=(750, 19, 1), data_format="channels_last"),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learn, beta_1=beta1, beta_2=beta2),
                 loss = tf.losses.SparseCategoricalCrossentropy(),
                 metrics = [tf.metrics.SparseCategoricalAccuracy()])
    return(model)

In [17]:
def fitModel(model, epoch):
    model.fit(train_dataset, epochs=epoch)
    return(model)

In [18]:
def testModel(model):
    metrics = model.evaluate(test_dataset)
    return(metrics)

In [19]:
rates = np.arange(0.01, 0.1, 0.01)
beta1s = np.arange(0.9, 0.99, 0.01)
beta2s = np.arange(0.99, 0.999, 0.001)

# learning rate does well around 0.01 or 0.02
# recently tried 0.005
# beta1 does well around 0.91
# beta2 does well around 0.997

# convergent to 1.0 at epoch 10 of
# 20e, 0.005lr, 0.91, 0.997

epochs = int(input("How many epochs? "))
rate = float(input("What's my learning rate? "))
beta1 = float(input("What's my beta1? "))
beta2 = float(input("What's my beta2? "))

myresults=[]

compiled = kerasModel(rate, beta1, beta2)
fitted = fitModel(compiled, epochs)
results = testModel(fitted)

KeyboardInterrupt: 

In [ ]:
print("Loss: ", results[0], "\nAccuracy: ", results[1])